In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from IPython.display import display, Markdown

import pandas as pd
import numpy as np

import json

np.__version__

'1.26.3'

# Pull in data

In [5]:
PREVIOUS_OUTPUT_PATH = './data/22-OUTPUT-gpt4-turbo-redo-uncategorized-and-no-thank-you.csv'
prev_df = pd.read_csv(PREVIOUS_OUTPUT_PATH)
prev_df

,performer,song,justification,love_song_sub_type
0,10cc,I'm Not In Love,"despite constant affirmations, the speaker den...",It's Complicated
1,3OH!3 Featuring Ke$ha,My First Kiss,"recounts the physical act of a first kiss, foc...",NaN
2,ABBA,Waterloo,depicts one's surrender to love as a moment of...,Serenade
3,AWB,Cut The Cake,uses wedding imagery as metaphor for a desirab...,NaN
4,Aaliyah,Back & Forth,focuses on the youthful experience of going ou...,NaN
...,...,...,...,...
5011,matchbox twenty,Unwell,"though the speaker does not feel well, he reas...",Courtship & Anticipation
5012,twenty one pilots,Heathens,"about social outcasts, not romance",NaN
5013,twenty one pilots,Ride,"introspective about life's journey, not focuse...",NaN
5014,twenty one pilots,Stressed Out,reflects on the pressures of adulthood and lon...,NaN


In [6]:
correction_df = pd.read_csv("./data/MJia hand fixing uncategorized love songs.csv")[['performer', 'song', 'justification', 'correction']]
correction_df["correction"] = correction_df.correction.replace({"(None)": np.NaN})
correction_df

,performer,song,justification,correction
0,3OH!3 Featuring Ke$ha,My First Kiss,"reminiscing about a first kiss, which is a fon...",Courtship & Anticipation
1,Anna Kendrick,Cups (Pitch Perfect's When I'm Gone),a song about leaving and moving on to pursue o...,Longing & Heartbreak
2,Ava Max,Sweet But Psycho,portrays a woman in love who is misunderstood ...,It's Complicated
3,Bruno Mars & Cardi B,Finesse,a song celebrating mutual attraction and good ...,Love Song for the Self
4,Calvin Harris,Summer,captures a summertime feeling of freedom and e...,It's Complicated
5,Calvin Harris Featuring Rihanna,This Is What You Came For,a party song about an electric connection with...,Courtship & Antisipation
6,Camila Cabello Featuring Young Thug,Havana,a narrative about being romantically entangled...,Longing & Heartbreak
7,Capital Cities,Safe And Sound,an upbeat melody offering support and security...,Serenade
8,Chris Brown,Don't Wake Me Up,talks about a dream of a perfect relationship ...,Serenade
9,Cobra Starship Featuring Sabi,You Make Me Feel...,about the exhilarating feeling someone gives y...,Courtship & Anticipation


# Insert MJia's corrections, isolate the non-hand-corrected uncategorized love songs (these need to be GPT-re-labeled)

In [168]:
merged_df = prev_df.merge(correction_df, on=['performer', 'song'], how='left', indicator=True)
filtered_prev_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

mjia_updated_df = pd.concat([filtered_prev_df, correction_df], ignore_index=True)
mjia_updated_df

# Overwrite values in love_song_sub_type column and justification_x column if there exists a value in the "correction" and justification_y columns, respectively
mjia_updated_df['love_song_sub_type'] = mjia_updated_df['love_song_sub_type'].fillna(mjia_updated_df['correction'])
mjia_updated_df['justification_x'] = mjia_updated_df['justification_x'].fillna(mjia_updated_df['justification'])
# Now drop all but performer, song, justification_x, and love_song_sub_type columns (and rename justification_x to justification)
mjia_updated_df = mjia_updated_df[['performer', 'song', 'justification_x', 'love_song_sub_type']].rename(columns={'justification_x': 'justification'})
mjia_updated_df


,performer,song,justification,love_song_sub_type
0,10cc,I'm Not In Love,"despite constant affirmations, the speaker den...",It's Complicated
1,ABBA,Waterloo,depicts one's surrender to love as a moment of...,Serenade
2,AWB,Cut The Cake,uses wedding imagery as metaphor for a desirab...,NaN
3,Aaliyah,Back & Forth,focuses on the youthful experience of going ou...,NaN
4,Aaron Neville,Everybody Plays The Fool,reminds the listener that everyone falls in lo...,Longing & Heartbreak
...,...,...,...,...
5011,P!nk,Try,about the inevitability of getting hurt when l...,It's Complicated
5012,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything,"about seizing the moment, with a blend of part...",Courtship & Anticipation
5013,Soko,We Might Be Dead By Tomorrow,emphasizing living in the current moment with ...,Longing & Heartbreak
5014,Taylor Swift,Blank Space,satirical view of the media's perception of he...,It's Complicated


In [25]:
PRIOR_OUTPUT_PATH = './data/17-OUTPUT-gpt4-turbo-redo-love-song-types-with-some-new-types.csv'

last_labeling_pass_with_extra_types = pd.read_csv(PRIOR_OUTPUT_PATH)
last_labeling_pass_with_extra_types

needs_relabel_df = last_labeling_pass_with_extra_types[
    last_labeling_pass_with_extra_types.love_song_sub_type.apply(
        lambda t: t == 'Uncategorized Love Song'
    )
]
# Also drop any rows whose performer, song is in the correction_df
needs_relabel_df = needs_relabel_df.merge(correction_df, on=['performer', 'song'], how='left', indicator=True)
needs_relabel_df = needs_relabel_df[needs_relabel_df['_merge'] == 'left_only'].drop(columns=['_merge'])


needs_relabel_df

,performer,song,justification_x,love_song_sub_type,justification_y,correction
0,10cc,I'm Not In Love,"the speaker claims he does not love her, but t...",Uncategorized Love Song,NaN,NaN
2,ABBA,Waterloo,uses the metaphor of the battle of Waterloo to...,Uncategorized Love Song,NaN,NaN
3,AWB,Cut The Cake,"seems to be about a celebration, possibly of l...",Uncategorized Love Song,NaN,NaN
4,Aaliyah,Back & Forth,"a song about going out to party, with a mild f...",Uncategorized Love Song,NaN,NaN
5,Aaron Neville,Everybody Plays The Fool,a song giving advice that sometimes love makes...,Uncategorized Love Song,NaN,NaN
...,...,...,...,...,...,...
207,U2,Mysterious Ways,touches on the enigmatic nature of a woman and...,Uncategorized Love Song,NaN,NaN
208,Will To Power,I'm Not In Love,claims not to be in love but the lyrics indica...,Uncategorized Love Song,NaN,NaN
209,William DeVaughn,Be Thankful For What You Got,"gratitude for what one has, including love, al...",Uncategorized Love Song,NaN,NaN
210,Wings,Silly Love Songs,"defends the validity and depth of love songs, ...",Uncategorized Love Song,NaN,NaN


In [41]:
# Print out a JSON string version of the dataframe corrections_df in the format:
# [[performer, song, justification, correction], ...]
# where justification and correction are both strings and np.na should be replaced with "NA"
# and where justification and correction are both strings and np.na should be replaced with "NA"
response_json_str = correction_df.fillna('NA')[['performer', 'song', 'justification', 'correction']].to_json(orient='values')
# Now remove the outer brackets:
response_json_str = response_json_str[1:-1]
response_json_str

'["3OH!3 Featuring Ke$ha","My First Kiss","reminiscing about a first kiss, which is a fond memory but not necessarily a song professing love","Courtship & Anticipation"],["Anna Kendrick","Cups (Pitch Perfect\'s When I\'m Gone)","a song about leaving and moving on to pursue one\'s dreams, ambiguous enough that it doesn\'t quite capture a specific romantic mood","Longing & Heartbreak"],["Ava Max","Sweet But Psycho","portrays a woman in love who is misunderstood to be \'psycho\'","It\'s Complicated"],["Bruno Mars & Cardi B","Finesse","a song celebrating mutual attraction and good times, though not directly focused on love or relationship depth","Love Song for the Self"],["Calvin Harris","Summer","captures a summertime feeling of freedom and excitement, but not expressly a love song","It\'s Complicated"],["Calvin Harris Featuring Rihanna","This Is What You Came For","a party song about an electric connection with someone, possibly romantic, but undefined","Courtship & Antisipation"],["Cami

In [43]:
# Print it again, but this time just with [[performer, song], ...]
# where performer and song are both strings
input_json_str = correction_df[['performer', 'song']].to_json(orient='values')
# Now remove the outer brackets:
input_json_str = input_json_str[1:-1]
input_json_str


'["3OH!3 Featuring Ke$ha","My First Kiss"],["Anna Kendrick","Cups (Pitch Perfect\'s When I\'m Gone)"],["Ava Max","Sweet But Psycho"],["Bruno Mars & Cardi B","Finesse"],["Calvin Harris","Summer"],["Calvin Harris Featuring Rihanna","This Is What You Came For"],["Camila Cabello Featuring Young Thug","Havana"],["Capital Cities","Safe And Sound"],["Chris Brown","Don\'t Wake Me Up"],["Cobra Starship Featuring Sabi","You Make Me Feel..."],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)"],["Fetty Wap","Trap Queen"],["Harry Styles","As It Was"],["Hozier","Take Me To Church"],["Katy Perry Featuring Kanye West","E.T."],["Kendrick Lamar & SZA","All The Stars"],["Lil Nas X","Panini"],["Lil Wayne","How To Love"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good"],["Owl City","Fireflies"],["P!nk","Try"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything"],["Soko","We Might Be Dead By Tomorrow"],["Taylor Swift","Blank Spa

# Fetch new labels using GPT4

In [31]:
import os
from openai import OpenAI

# First, activate the './.env' file which sets the OPENAI_API_KEY environment variable using python-dotenv
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [32]:
import ast

def eval_this_str(s):
    return ast.literal_eval(s)

In [58]:
LOVE_SONG_LABELING_PROMPT = """
You are an expert pop music critic, able to label pop songs as different kinds of love songs using subtle context, emotions, & relational implications.

For each Billboard Top 10 hit in the form [song, performer] in the larger input array, I want you to respond with a valid JSON array of:

[performer, song, justification, love_song_subtype_label]

The justification is a short snippet explaining why the song best fits in the chosen category.

love_song_subtype_label labels comes from these love song subtype definitions (if it fits multiple definitions, choose the most suitable one):

"Serenade": A song directed at someone the singer loves, expressing their deep affection, undying love, infatuation, or praising someone as a soulmate.

"Sexual Conquest": Songs that are less about the object of affection and more about the singer's own prowess or success in romantic pursuits.

"Love Song for the Self": Songs that express self-love and affirmation, whether for one's character or physical qualities, sometimes following a period of growth or learning from past relationships.

"Good Riddance": typically after a breakup with someone who now seems unsuitable. songs about having had someone's love, but being glad now you've moved on (or realizing you should), often involving self-empowerment, anger, revenge, relief, or bitterness, often because of betrayal or potentially cheating.

"Longing & Heartbreak": songs where you love or miss someone, but they are unable to return your love. Either because you broke up and the relationship is over, or because you were never together and the love was unrequited. COuld be full of regret, longing, frustration, sadness, hopelessness, even grief.

"Courtship & Anticipation": the speaker is courting or seeking someone's romantic or sexual love, but they don't have it yet. The love interest does not yet know the speaker well, but the speaker hopes they will want to get to know them better and or please them the way they want.

"It's Complicated": Songs about complicated relationships, often with conflicting emotions like love, hate, frustration, and desire intermingling, whether light-hearted or profound in tone.

"Platonic Love": Songs about non-romantic yet strong and meaningful relationships, such as friendships or familial bonds.

"NA": song is not a love song at all. This would include any non-love song, but also instrumentals, for example.

Here's an example input:

```json
[
	["'N Sync", "This I Promise You"],
	["John Legend", "All of Me"],
	["Beyonce", "Halo"],
	["T-Pain Featuring Lil Wayne", "Can't Believe It"],
	["Adele", "Someone Like You"],
	["Mariah Carey", "Without You"],
	["Dolly Parton", "Jolene"],
	["Bill Withers", "Ain't No Sunshine"],
	["The Rolling Stones", "Miss You"],
	["Whitney Houston", "I Will Always Love You"],
	["Taylor Swift", "Teardrops on My Guitar"],
	["Lesley Gore", "She's A Fool"],
	["Taylor Swift", "You Belong With Me"],
	["Justin Timberlake", "Suit & Tie"],
	["T-Pain Featuring Mike Jones", "I'm N Luv (Wit A Stripper)"],
	["Rihanna", "Only Girl (In the World)"],
	["Britney Spears", "Hold It Against Me"],
	["T-Pain", "Buy U A Drank (Shawty Snappin')"],
	["2Pac", "Dear Mama"],
	["Saweetie", "Best Friend (feat. Doja Cat)"],
	["James Taylor", "You've Got a Friend"],
	["Bill Withers", "Lean On Me"],
	["Little Mix and Jason Derulo", "Secret Love Song"],
	["Katy Perry", "Hot N Cold"],
	["Pat Benatar", "Love Is a Battlefield"],
	["Lauryn Hill", "Ex-Factor"],
	["24kGoldn", "Mood"],
	["T-Pain", "I'm Sprung"],
	["Kelly Clarkson", "Since U Been Gone"],
	["Blu Cantrell", "Hit 'Em Up Style (Oops!)"],
	["Selena Gomez", "Look at Her Now"],
	["Taylor Swift", "Picture To Burn"],
	["Christina Aguilera", "Fighter"],
	["Jamie Foxx Featuring T-Pain", "Blame It"],
	["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
	["The Beach Boys", "I Get Around"],
	["Robin Thicke ft. Pharrell & T.I.", "Blurred Lines"],
	["112", "Peaches & Cream"],
	["Lady Gaga", "Poker Face"],
	["Ariana Grande", "Thank U, Next"],
	["Lizzo", "Good As Hell"],
	["Christina Aguilera", "Beautiful"],
	["Britney Spears", "Womanizer"],
	["TLC", "No Scrubs"],
	["Meghan Trainor", "NO"],
	["Radiohead", "Creep"],
	["Rick Springfield", "Jessie's Girl"],
	["21 Savage & Metro Boomin", "Runnin"],
	["twenty one pilots", "Heathens"],
	["will.i.am & Britney Spears", "Scream & Shout"],
    ["3OH!3 Featuring Ke$ha","My First Kiss"],["Anna Kendrick","Cups (Pitch Perfect\'s When I\'m Gone)"],["Ava Max","Sweet But Psycho"],["Bruno Mars & Cardi B","Finesse"],["Calvin Harris","Summer"],["Calvin Harris Featuring Rihanna","This Is What You Came For"],["Camila Cabello Featuring Young Thug","Havana"],["Capital Cities","Safe And Sound"],["Chris Brown","Don\'t Wake Me Up"],["Cobra Starship Featuring Sabi","You Make Me Feel..."],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)"],["Fetty Wap","Trap Queen"],["Harry Styles","As It Was"],["Hozier","Take Me To Church"],["Katy Perry Featuring Kanye West","E.T."],["Kendrick Lamar & SZA","All The Stars"],["Lil Nas X","Panini"],["Lil Wayne","How To Love"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good"],["Owl City","Fireflies"],["P!nk","Try"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything"],["Soko","We Might Be Dead By Tomorrow"],["Taylor Swift","Blank Space"],["The Chainsmokers","Paris"]

]
```

Here's the sample corresponding output:

```json
{"output":[
    ["'N Sync","This I Promise You","promising to always love someone","Serenade"],["John Legend","All of Me","praises lover, who's affection makes any struggle worth it, promises to give all of them","Serenade"],["Beyonce","Halo","celebrates the angelic quality of a lover and testifies to the positive changes as a result of this love","Serenade"],["T-Pain Featuring Lil Wayne","Can't Believe It","praises love interest, telling her of all the things he'll do for her","Serenade"],["Adele","Someone Like You","bemoans that a lover has moved on to a happy life with another partner while the speaker struggles to move on","Longing & Heartbreak"],["Mariah Carey","Without You","at the moment of a breakup, she expresses despair over everything she will lose","Longing & Heartbreak"],["Dolly Parton","Jolene","her heart is breaking because she fears her male lover will be easily stolen by another woman","Longing & Heartbreak"],["Bill Withers","Ain't No Sunshine","when a lover is away, he expresses that his world seems to become darker","Longing & Heartbreak"],["The Rolling Stones","Miss You","expresses how deeply he misses the lover, and wishes he didn't have to wait","Longing & Heartbreak"],["Whitney Houston","I Will Always Love You","wishes her now gone lover well, reminisces, will never forget","Longing & Heartbreak"],["Taylor Swift","Teardrops on My Guitar","she's in love with a friend who's in love with someone else, she has to pretend she doesn't love him but secretly she's suffering","Longing & Heartbreak"],["Lesley Gore","She's A Fool","her love is with another woman who doesn't appreciate his virtues, longs to some day be noticed by him","Longing & Heartbreak"],["Taylor Swift","You Belong With Me","she's making an empassioned argument that her love object should be with her instead of who he's currently with","Courtship & Anticipation"],["Justin Timberlake","Suit & Tie","looking forward to a night of dancing with his beloved","Courtship & Anticipation"],["T-Pain Featuring Mike Jones","I'm N Luv (Wit A Stripper)","he has strong romantic and sexual feelings for a stripper who doesn't yet know him personally","Courtship & Anticipation"],["Rihanna","Only Girl (In the World)","telling her new love how she wants him to make her feel","Courtship & Anticipation"],["Britney Spears","Hold It Against Me","she's interested in a stragner and approaches them hoping to see if they might get together","Courtship & Anticipation"],["T-Pain","Buy U A Drank (Shawty Snappin')","he spots a woman at the bar and offers to buy her a drink as a prelude to potentially getting together","Courtship & Anticipation"],["2Pac","Dear Mama","love and appreciation for his mother","Platonic Love"],["Saweetie","Best Friend (feat. Doja Cat)","boasts about a close friend who is succesful and strong","Platonic Love"],["James Taylor","You've Got a Friend","promises a close friend they'll be there thick and thin","Platonic Love"],["Bill Withers","Lean On Me","invites a friend to rely on him when they are struggling, as everyone needs platonic support at times","Platonic Love"],["Little Mix and Jason Derulo","Secret Love Song","the speakers long to be united with their lover in public, but must remain in secret","It's Complicated"],["Katy Perry","Hot N Cold","about navigating a lover who cannot make up their mind and is inconsistent","It's Complicated"],["Pat Benatar","Love Is a Battlefield","the relationship she is in alternates between being loving and painful and she doesn't feel like she can leave","It's Complicated"],["Lauryn Hill","Ex-Factor","she feels trapped in a relationship where she feels she's giving more than she's recieving, but whenever she tries to leave, she can't","It's Complicated"],["24kGoldn","Mood","bemoans why his female love interest is always difficult, moody, and inconsistent","It's Complicated"],["T-Pain","I'm Sprung","knows his love interest doesn't deserve him, yet he can't stay away, he vascilates between feelings of deep love and wanting to get away as fast as possible","It's Complicated"],["Kelly Clarkson","Since U Been Gone","ending a bad relationship leaves her feeling better and more in control","Good Riddance"],["Blu Cantrell","Hit 'Em Up Style (Oops!)","reacts to being cheated on by gleefully spending her ex-lover's money, as an act or revents","Good Riddance"],["Selena Gomez","Look at Her Now","after being cheated on, she has now moved on and is better than ever","Good Riddance"],["Taylor Swift","Picture To Burn","after realizing how self-centered and unsuitable her ex-lover is, she vents about all of his negative qualities","Good Riddance"],["Christina Aguilera","Fighter","she thanks the lover because his abuse actually ended up making her stronger and wiser","Good Riddance"],["Jamie Foxx Featuring T-Pain","Blame It","speaker brags about his sexual power & alure, confident his love interest will want to hook up with him","Sexual Conquest"],["6ix9ine Featuring Nicki Minaj & Murda Beatz","FEFE","about sexual prowess with many sexual references","Sexual Conquest"],["The Beach Boys","I Get Around","brag about constantly seeking out new women and never being rejected","Sexual Conquest"],["Robin Thicke ft. Pharrell & T.I.","Blurred Lines","speaker brags about being about to 'get nasty' with a good girl who he claims wants sex","Sexual Conquest"],["112","Peaches & Cream","about sex & being addicted to a tasty woman","Sexual Conquest"],["Lady Gaga","Poker Face","brags about a male sex partner who does not know she is sleeping with a women as well","Sexual Conquest"],["Ariana Grande","Thank U, Next","brags about being unfased and even improved by past relationships ending, and in fact is now thriving in life","Love Song for the Self"],["Lizzo","Good As Hell","an anthem for leaving behidn negativity or men who don't love you anymore, and instead enjoying who you are with pride","Love Song for the Self"],["Christina Aguilera","Beautiful","an anthem proclaiming ones beauty, even in the face of deragatory words or spiteful ex-lovers","Love Song for the Self"],["Britney Spears","Womanizer","in her prowess, she recognizes a womanizer and rejects his advances","Love Song for the Self"],["TLC","No Scrubs","speakers asserts that they will reject so-called scrubs because she wants a higher quality man","Love Song for the Self"],["Meghan Trainor","NO","an anthem for women to confidently reject unwanted male advances, because the men aren't needed for the women to be in the zone","Love Song for the Self"],["Radiohead","Creep","in contrast to his love interest's angelic nature, he feels ugly and invisible, and hates on himself, which does not fit in Longing & Hearbreak or even Good Riddance","Longing & Heartbreak"],["Rick Springfield","Jessie's Girl","praises his friends lover and expresses he's confused she isn't into him too, and he's jealous: themes to unusual to fit into the given love song subtypes.","It's Complicated"],["21 Savage & Metro Boomin","Runnin","a gangster rap, not romance","NA"],["twenty one pilots","Heathens","about social outcasts, not romance","NA"],["will.i.am & Britney Spears","Scream & Shout","about dancing in the club, with no real elements of romance directed toward a specific love interest","NA"],
    ["3OH!3 Featuring Ke$ha","My First Kiss","reminiscing about a first kiss, which is a fond memory but not necessarily a song professing love","Courtship & Anticipation"],["Anna Kendrick","Cups (Pitch Perfect\'s When I\'m Gone)","a song about leaving and moving on to pursue one\'s dreams, ambiguous enough that it doesn\'t quite capture a specific romantic mood","Longing & Heartbreak"],["Ava Max","Sweet But Psycho","portrays a woman in love who is misunderstood to be \'psycho\'","It\'s Complicated"],["Bruno Mars & Cardi B","Finesse","a song celebrating mutual attraction and good times, though not directly focused on love or relationship depth","Love Song for the Self"],["Calvin Harris","Summer","captures a summertime feeling of freedom and excitement, but not expressly a love song","It\'s Complicated"],["Calvin Harris Featuring Rihanna","This Is What You Came For","a party song about an electric connection with someone, possibly romantic, but undefined","Courtship & Antisipation"],["Camila Cabello Featuring Young Thug","Havana","a narrative about being romantically entangled with a suave character named \'East Atlanta\', though not deeply emotional","Longing & Heartbreak"],["Capital Cities","Safe And Sound","an upbeat melody offering support and security, can be interpreted as platonic or romantic","Serenade"],["Chris Brown","Don\'t Wake Me Up","talks about a dream of a perfect relationship he doesn\'t want to end","Serenade"],["Cobra Starship Featuring Sabi","You Make Me Feel...","about the exhilarating feeling someone gives you, could be interpreted as a romantic song","Courtship & Anticipation"],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)","mixes lyrics from previous Elton John\'s songs, dealing with longing for warmth in a cold-hearted world","NA"],["Fetty Wap","Trap Queen","about a woman helping her man in his drug business, love within a criminal context","Serenade"],["Harry Styles","As It Was","reflects on changes and how someone fits into the singer\'s life","NA"],["Hozier","Take Me To Church","metaphorically compares loving a person to religion, and critiques institutions that oppose his love","Serenade"],["Katy Perry Featuring Kanye West","E.T.","compares a lover to an alien, otherworldly love interest","Courtship & Anticipation"],["Kendrick Lamar & SZA","All The Stars","mentions a relationship but in the context of larger themes of life and legacy","NA"],["Lil Nas X","Panini","about fans\' attachment and his own personal journey, not directly a love song","NA"],["Lil Wayne","How To Love","a narrative about a woman\\u2019s life and her struggle to understand love","Courtship & Anticipation"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa","describes using women to set up a robbery, trust and love are manipulated for personal gain, which doesn\\u2019t fit cleanly into defined love song subcategories","Good Riddance"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good","remembers when love was good and pure, sort of a celebration of past love","Serenade"],["Owl City","Fireflies","about wonder at the world more than a love song, though could suggest a sense of platonic connection","NA"],["P!nk","Try","about the inevitability of getting hurt when looking for love but encourages to try again","It\'s Complicated"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything","about seizing the moment, with a blend of party and romantic elements","Courtship & Anticipation"],["Soko","We Might Be Dead By Tomorrow","emphasizing living in the current moment with the person they care about, has a romantic undertone","Longing & Heartbreak"],["Taylor Swift","Blank Space","satirical view of the media\'s perception of her dating life, embodying a character","It\'s Complicated"],["The Chainsmokers","Paris","reflects on an escape with a partner, hinting at a deeper connection but not overtly romantic","Serenade"]
    ]}
```

Ok, now generate the output for this input:


"""

In [59]:
# pretty print the JSON string
display(Markdown(LOVE_SONG_LABELING_PROMPT))
# LOVE_SONG_LABELING_PROMPT


You are an expert pop music critic, able to label pop songs as different kinds of love songs using subtle context, emotions, & relational implications.

For each Billboard Top 10 hit in the form [song, performer] in the larger input array, I want you to respond with a valid JSON array of:

[performer, song, justification, love_song_subtype_label]

The justification is a short snippet explaining why the song best fits in the chosen category.

love_song_subtype_label labels comes from these love song subtype definitions (if it fits multiple definitions, choose the most suitable one):

"Serenade": A song directed at someone the singer loves, expressing their deep affection, undying love, infatuation, or praising someone as a soulmate.

"Sexual Conquest": Songs that are less about the object of affection and more about the singer's own prowess or success in romantic pursuits.

"Love Song for the Self": Songs that express self-love and affirmation, whether for one's character or physical qualities, sometimes following a period of growth or learning from past relationships.

"Good Riddance": typically after a breakup with someone who now seems unsuitable. songs about having had someone's love, but being glad now you've moved on (or realizing you should), often involving self-empowerment, anger, revenge, relief, or bitterness, often because of betrayal or potentially cheating.

"Longing & Heartbreak": songs where you love or miss someone, but they are unable to return your love. Either because you broke up and the relationship is over, or because you were never together and the love was unrequited. COuld be full of regret, longing, frustration, sadness, hopelessness, even grief.

"Courtship & Anticipation": the speaker is courting or seeking someone's romantic or sexual love, but they don't have it yet. The love interest does not yet know the speaker well, but the speaker hopes they will want to get to know them better and or please them the way they want.

"It's Complicated": Songs about complicated relationships, often with conflicting emotions like love, hate, frustration, and desire intermingling, whether light-hearted or profound in tone.

"Platonic Love": Songs about non-romantic yet strong and meaningful relationships, such as friendships or familial bonds.

"NA": song is not a love song at all. This would include any non-love song, but also instrumentals, for example.

Here's an example input:

```json
[
	["'N Sync", "This I Promise You"],
	["John Legend", "All of Me"],
	["Beyonce", "Halo"],
	["T-Pain Featuring Lil Wayne", "Can't Believe It"],
	["Adele", "Someone Like You"],
	["Mariah Carey", "Without You"],
	["Dolly Parton", "Jolene"],
	["Bill Withers", "Ain't No Sunshine"],
	["The Rolling Stones", "Miss You"],
	["Whitney Houston", "I Will Always Love You"],
	["Taylor Swift", "Teardrops on My Guitar"],
	["Lesley Gore", "She's A Fool"],
	["Taylor Swift", "You Belong With Me"],
	["Justin Timberlake", "Suit & Tie"],
	["T-Pain Featuring Mike Jones", "I'm N Luv (Wit A Stripper)"],
	["Rihanna", "Only Girl (In the World)"],
	["Britney Spears", "Hold It Against Me"],
	["T-Pain", "Buy U A Drank (Shawty Snappin')"],
	["2Pac", "Dear Mama"],
	["Saweetie", "Best Friend (feat. Doja Cat)"],
	["James Taylor", "You've Got a Friend"],
	["Bill Withers", "Lean On Me"],
	["Little Mix and Jason Derulo", "Secret Love Song"],
	["Katy Perry", "Hot N Cold"],
	["Pat Benatar", "Love Is a Battlefield"],
	["Lauryn Hill", "Ex-Factor"],
	["24kGoldn", "Mood"],
	["T-Pain", "I'm Sprung"],
	["Kelly Clarkson", "Since U Been Gone"],
	["Blu Cantrell", "Hit 'Em Up Style (Oops!)"],
	["Selena Gomez", "Look at Her Now"],
	["Taylor Swift", "Picture To Burn"],
	["Christina Aguilera", "Fighter"],
	["Jamie Foxx Featuring T-Pain", "Blame It"],
	["6ix9ine Featuring Nicki Minaj & Murda Beatz", "FEFE"],
	["The Beach Boys", "I Get Around"],
	["Robin Thicke ft. Pharrell & T.I.", "Blurred Lines"],
	["112", "Peaches & Cream"],
	["Lady Gaga", "Poker Face"],
	["Ariana Grande", "Thank U, Next"],
	["Lizzo", "Good As Hell"],
	["Christina Aguilera", "Beautiful"],
	["Britney Spears", "Womanizer"],
	["TLC", "No Scrubs"],
	["Meghan Trainor", "NO"],
	["Radiohead", "Creep"],
	["Rick Springfield", "Jessie's Girl"],
	["21 Savage & Metro Boomin", "Runnin"],
	["twenty one pilots", "Heathens"],
	["will.i.am & Britney Spears", "Scream & Shout"],
    ["3OH!3 Featuring Ke$ha","My First Kiss"],["Anna Kendrick","Cups (Pitch Perfect's When I'm Gone)"],["Ava Max","Sweet But Psycho"],["Bruno Mars & Cardi B","Finesse"],["Calvin Harris","Summer"],["Calvin Harris Featuring Rihanna","This Is What You Came For"],["Camila Cabello Featuring Young Thug","Havana"],["Capital Cities","Safe And Sound"],["Chris Brown","Don't Wake Me Up"],["Cobra Starship Featuring Sabi","You Make Me Feel..."],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)"],["Fetty Wap","Trap Queen"],["Harry Styles","As It Was"],["Hozier","Take Me To Church"],["Katy Perry Featuring Kanye West","E.T."],["Kendrick Lamar & SZA","All The Stars"],["Lil Nas X","Panini"],["Lil Wayne","How To Love"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good"],["Owl City","Fireflies"],["P!nk","Try"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything"],["Soko","We Might Be Dead By Tomorrow"],["Taylor Swift","Blank Space"],["The Chainsmokers","Paris"]

]
```

Here's the sample corresponding output:

```json
{"output":[
    ["'N Sync","This I Promise You","promising to always love someone","Serenade"],["John Legend","All of Me","praises lover, who's affection makes any struggle worth it, promises to give all of them","Serenade"],["Beyonce","Halo","celebrates the angelic quality of a lover and testifies to the positive changes as a result of this love","Serenade"],["T-Pain Featuring Lil Wayne","Can't Believe It","praises love interest, telling her of all the things he'll do for her","Serenade"],["Adele","Someone Like You","bemoans that a lover has moved on to a happy life with another partner while the speaker struggles to move on","Longing & Heartbreak"],["Mariah Carey","Without You","at the moment of a breakup, she expresses despair over everything she will lose","Longing & Heartbreak"],["Dolly Parton","Jolene","her heart is breaking because she fears her male lover will be easily stolen by another woman","Longing & Heartbreak"],["Bill Withers","Ain't No Sunshine","when a lover is away, he expresses that his world seems to become darker","Longing & Heartbreak"],["The Rolling Stones","Miss You","expresses how deeply he misses the lover, and wishes he didn't have to wait","Longing & Heartbreak"],["Whitney Houston","I Will Always Love You","wishes her now gone lover well, reminisces, will never forget","Longing & Heartbreak"],["Taylor Swift","Teardrops on My Guitar","she's in love with a friend who's in love with someone else, she has to pretend she doesn't love him but secretly she's suffering","Longing & Heartbreak"],["Lesley Gore","She's A Fool","her love is with another woman who doesn't appreciate his virtues, longs to some day be noticed by him","Longing & Heartbreak"],["Taylor Swift","You Belong With Me","she's making an empassioned argument that her love object should be with her instead of who he's currently with","Courtship & Anticipation"],["Justin Timberlake","Suit & Tie","looking forward to a night of dancing with his beloved","Courtship & Anticipation"],["T-Pain Featuring Mike Jones","I'm N Luv (Wit A Stripper)","he has strong romantic and sexual feelings for a stripper who doesn't yet know him personally","Courtship & Anticipation"],["Rihanna","Only Girl (In the World)","telling her new love how she wants him to make her feel","Courtship & Anticipation"],["Britney Spears","Hold It Against Me","she's interested in a stragner and approaches them hoping to see if they might get together","Courtship & Anticipation"],["T-Pain","Buy U A Drank (Shawty Snappin')","he spots a woman at the bar and offers to buy her a drink as a prelude to potentially getting together","Courtship & Anticipation"],["2Pac","Dear Mama","love and appreciation for his mother","Platonic Love"],["Saweetie","Best Friend (feat. Doja Cat)","boasts about a close friend who is succesful and strong","Platonic Love"],["James Taylor","You've Got a Friend","promises a close friend they'll be there thick and thin","Platonic Love"],["Bill Withers","Lean On Me","invites a friend to rely on him when they are struggling, as everyone needs platonic support at times","Platonic Love"],["Little Mix and Jason Derulo","Secret Love Song","the speakers long to be united with their lover in public, but must remain in secret","It's Complicated"],["Katy Perry","Hot N Cold","about navigating a lover who cannot make up their mind and is inconsistent","It's Complicated"],["Pat Benatar","Love Is a Battlefield","the relationship she is in alternates between being loving and painful and she doesn't feel like she can leave","It's Complicated"],["Lauryn Hill","Ex-Factor","she feels trapped in a relationship where she feels she's giving more than she's recieving, but whenever she tries to leave, she can't","It's Complicated"],["24kGoldn","Mood","bemoans why his female love interest is always difficult, moody, and inconsistent","It's Complicated"],["T-Pain","I'm Sprung","knows his love interest doesn't deserve him, yet he can't stay away, he vascilates between feelings of deep love and wanting to get away as fast as possible","It's Complicated"],["Kelly Clarkson","Since U Been Gone","ending a bad relationship leaves her feeling better and more in control","Good Riddance"],["Blu Cantrell","Hit 'Em Up Style (Oops!)","reacts to being cheated on by gleefully spending her ex-lover's money, as an act or revents","Good Riddance"],["Selena Gomez","Look at Her Now","after being cheated on, she has now moved on and is better than ever","Good Riddance"],["Taylor Swift","Picture To Burn","after realizing how self-centered and unsuitable her ex-lover is, she vents about all of his negative qualities","Good Riddance"],["Christina Aguilera","Fighter","she thanks the lover because his abuse actually ended up making her stronger and wiser","Good Riddance"],["Jamie Foxx Featuring T-Pain","Blame It","speaker brags about his sexual power & alure, confident his love interest will want to hook up with him","Sexual Conquest"],["6ix9ine Featuring Nicki Minaj & Murda Beatz","FEFE","about sexual prowess with many sexual references","Sexual Conquest"],["The Beach Boys","I Get Around","brag about constantly seeking out new women and never being rejected","Sexual Conquest"],["Robin Thicke ft. Pharrell & T.I.","Blurred Lines","speaker brags about being about to 'get nasty' with a good girl who he claims wants sex","Sexual Conquest"],["112","Peaches & Cream","about sex & being addicted to a tasty woman","Sexual Conquest"],["Lady Gaga","Poker Face","brags about a male sex partner who does not know she is sleeping with a women as well","Sexual Conquest"],["Ariana Grande","Thank U, Next","brags about being unfased and even improved by past relationships ending, and in fact is now thriving in life","Love Song for the Self"],["Lizzo","Good As Hell","an anthem for leaving behidn negativity or men who don't love you anymore, and instead enjoying who you are with pride","Love Song for the Self"],["Christina Aguilera","Beautiful","an anthem proclaiming ones beauty, even in the face of deragatory words or spiteful ex-lovers","Love Song for the Self"],["Britney Spears","Womanizer","in her prowess, she recognizes a womanizer and rejects his advances","Love Song for the Self"],["TLC","No Scrubs","speakers asserts that they will reject so-called scrubs because she wants a higher quality man","Love Song for the Self"],["Meghan Trainor","NO","an anthem for women to confidently reject unwanted male advances, because the men aren't needed for the women to be in the zone","Love Song for the Self"],["Radiohead","Creep","in contrast to his love interest's angelic nature, he feels ugly and invisible, and hates on himself, which does not fit in Longing & Hearbreak or even Good Riddance","Longing & Heartbreak"],["Rick Springfield","Jessie's Girl","praises his friends lover and expresses he's confused she isn't into him too, and he's jealous: themes to unusual to fit into the given love song subtypes.","It's Complicated"],["21 Savage & Metro Boomin","Runnin","a gangster rap, not romance","NA"],["twenty one pilots","Heathens","about social outcasts, not romance","NA"],["will.i.am & Britney Spears","Scream & Shout","about dancing in the club, with no real elements of romance directed toward a specific love interest","NA"],
    ["3OH!3 Featuring Ke$ha","My First Kiss","reminiscing about a first kiss, which is a fond memory but not necessarily a song professing love","Courtship & Anticipation"],["Anna Kendrick","Cups (Pitch Perfect's When I'm Gone)","a song about leaving and moving on to pursue one's dreams, ambiguous enough that it doesn't quite capture a specific romantic mood","Longing & Heartbreak"],["Ava Max","Sweet But Psycho","portrays a woman in love who is misunderstood to be 'psycho'","It's Complicated"],["Bruno Mars & Cardi B","Finesse","a song celebrating mutual attraction and good times, though not directly focused on love or relationship depth","Love Song for the Self"],["Calvin Harris","Summer","captures a summertime feeling of freedom and excitement, but not expressly a love song","It's Complicated"],["Calvin Harris Featuring Rihanna","This Is What You Came For","a party song about an electric connection with someone, possibly romantic, but undefined","Courtship & Antisipation"],["Camila Cabello Featuring Young Thug","Havana","a narrative about being romantically entangled with a suave character named 'East Atlanta', though not deeply emotional","Longing & Heartbreak"],["Capital Cities","Safe And Sound","an upbeat melody offering support and security, can be interpreted as platonic or romantic","Serenade"],["Chris Brown","Don't Wake Me Up","talks about a dream of a perfect relationship he doesn't want to end","Serenade"],["Cobra Starship Featuring Sabi","You Make Me Feel...","about the exhilarating feeling someone gives you, could be interpreted as a romantic song","Courtship & Anticipation"],["Elton John & Dua Lipa","Cold Heart (PNAU Remix)","mixes lyrics from previous Elton John's songs, dealing with longing for warmth in a cold-hearted world","NA"],["Fetty Wap","Trap Queen","about a woman helping her man in his drug business, love within a criminal context","Serenade"],["Harry Styles","As It Was","reflects on changes and how someone fits into the singer's life","NA"],["Hozier","Take Me To Church","metaphorically compares loving a person to religion, and critiques institutions that oppose his love","Serenade"],["Katy Perry Featuring Kanye West","E.T.","compares a lover to an alien, otherworldly love interest","Courtship & Anticipation"],["Kendrick Lamar & SZA","All The Stars","mentions a relationship but in the context of larger themes of life and legacy","NA"],["Lil Nas X","Panini","about fans' attachment and his own personal journey, not directly a love song","NA"],["Lil Wayne","How To Love","a narrative about a woman\u2019s life and her struggle to understand love","Courtship & Anticipation"],["Lil Wayne Featuring Kendrick Lamar","Mona Lisa","describes using women to set up a robbery, trust and love are manipulated for personal gain, which doesn\u2019t fit cleanly into defined love song subcategories","Good Riddance"],["Michael Jackson & Justin Timberlake","Love Never Felt So Good","remembers when love was good and pure, sort of a celebration of past love","Serenade"],["Owl City","Fireflies","about wonder at the world more than a love song, though could suggest a sense of platonic connection","NA"],["P!nk","Try","about the inevitability of getting hurt when looking for love but encourages to try again","It's Complicated"],["Pitbull Featuring Ne-Yo, Afrojack & Nayer","Give Me Everything","about seizing the moment, with a blend of party and romantic elements","Courtship & Anticipation"],["Soko","We Might Be Dead By Tomorrow","emphasizing living in the current moment with the person they care about, has a romantic undertone","Longing & Heartbreak"],["Taylor Swift","Blank Space","satirical view of the media's perception of her dating life, embodying a character","It's Complicated"],["The Chainsmokers","Paris","reflects on an escape with a partner, hinting at a deeper connection but not overtly romantic","Serenade"]
    ]}
```

Ok, now generate the output for this input:




In [ ]:
def get_love_song_labels_for_rows(row_subset_str):
    try: 
        completion = client.chat.completions.create(
          model="gpt-4-1106-preview",
          response_format={'type': "json_object"},
          messages=[
            {"role": "user", "content": LOVE_SONG_LABELING_PROMPT},
            {"role": "user", "content": row_subset_str},
          ]
        )
        content = completion.choices[0].message.content
        # return content
        response_content = eval_this_str(content)['output']
        # print(f"Response contains {completion['usage']['completion_tokens']} tokens out of 4096 maximum allowed tokens.")
        return response_content
    except Exception as e:
        print('ERROR FETCHING', e)
        return []

### Fetch in batches

In [ ]:

def fetch_all_data_in_batches(df, increment=25):
    gpt_labeled_result = []
    for i in range(101):
        subset = df[['performer', 'song']][i*increment : (i+1)*increment]
        if len(subset):
            subset_as_json_string = subset.to_json(orient='values')
            print([i*increment, (i+1)*increment])
            print(subset_as_json_string, '\n')
            fetched_and_formatted_res = get_love_song_labels_for_rows(subset_as_json_string)
            print(fetched_and_formatted_res)
            gpt_labeled_result = [*gpt_labeled_result, *fetched_and_formatted_res]
    return gpt_labeled_result

In [ ]:
def get_file_path(postfix):
    return f'./data/22-TEMP-gpt4-re-relabel-love-song-types-{postfix}.csv'

In [ ]:
def save_to_temp_csv(gpt_labeled_result, run_num):
    first_label_pass_df = pd.DataFrame(
        gpt_labeled_result, 
        columns=['performer', 'song', 
                 'justification',
                 'love_song_sub_type']
    )
    first_label_pass_df.to_csv(get_file_path(run_num), index=False)

# (run a test call to be sure things look ok)
For 100 rows at once:
For GPT4 turbo: 3min, about 3400 tokens out of 4096 maximum allowed tokens 

In [ ]:
%%time
TEST_POSTFIX = 'JUST A TEST, DELETE ME'
save_to_temp_csv(
    fetch_all_data_in_batches(
        needs_relabel_df.iloc[0:2]
    ), 
    TEST_POSTFIX
)

[0, 75]
[["10cc","I'm Not In Love"],["ABBA","Waterloo"]] 

[['10cc', "I'm Not In Love", "The song narrates a denial of one's own feelings, pretending not to be in love, which is a complicated emotional state", "It's Complicated"], ['ABBA', 'Waterloo', 'The song compares falling in love to being defeated in battle, and surrendering to love despite attempts at resistance', 'Serenade']]
CPU times: user 27.8 ms, sys: 2.77 ms, total: 30.6 ms
Wall time: 8.55 s


In [155]:
pd.read_csv(get_file_path(TEST_POSTFIX))#.iloc[1].justification

,performer,song,justification,love_song_sub_type
0,10cc,I'm Not In Love,The song narrates a denial of one's own feelin...,It's Complicated
1,ABBA,Waterloo,The song compares falling in love to being def...,Serenade


# Ok, run the actual fetch

In [161]:
# save_to_temp_csv(
#     fetch_all_data_in_batches(
#         needs_relabel_df.iloc[0:1000],
#         20
#     ), 
#     1
# )

# Process results

In [160]:
love_song_subtypes_df = pd.concat([
    pd.read_csv(get_file_path(1)),
])
love_song_subtypes_df = love_song_subtypes_df.drop_duplicates(subset=['performer', 'song'])
love_song_subtypes_df

,performer,song,justification,love_song_sub_type
0,10cc,I'm Not In Love,"a repeated denial of being in love, which sugg...",It's Complicated
1,ABBA,Waterloo,"compares being in love with surrender, as if l...",Serenade
2,AWB,Cut The Cake,symbolically about marriage but focuses on the...,NaN
3,Aaliyah,Back & Forth,"a song about having a good time, but not a spe...",NaN
4,Aaron Neville,Everybody Plays The Fool,advising that everyone will be the fool in lov...,Longing & Heartbreak
...,...,...,...,...
181,U2,Mysterious Ways,illustrates how a woman can change a man but d...,It's Complicated
182,Will To Power,I'm Not In Love,"despite suggestions otherwise, the song's refr...",Good Riddance
183,William DeVaughn,Be Thankful For What You Got,more about life's blessings and success than r...,NaN
184,Wings,Silly Love Songs,"a commentary on and celebration of love songs,...",Serenade


In [162]:
love_song_subtypes_df.love_song_sub_type.value_counts()

love_song_sub_type
Serenade                    50
It's Complicated            28
Longing & Heartbreak        19
Courtship & Anticipation    17
Platonic Love                7
Sexual Conquest              6
Love Song for the Self       2
Good Riddance                2
Name: count, dtype: int64

# Ok, now merge it back in & export

In [167]:
mjia_updated_df

,performer,song,justification_x,love_song_sub_type,justification_y,correction,justification
0,10cc,I'm Not In Love,"despite constant affirmations, the speaker den...",It's Complicated,NaN,NaN,NaN
1,ABBA,Waterloo,depicts one's surrender to love as a moment of...,Serenade,NaN,NaN,NaN
2,AWB,Cut The Cake,uses wedding imagery as metaphor for a desirab...,NaN,NaN,NaN,NaN
3,Aaliyah,Back & Forth,focuses on the youthful experience of going ou...,NaN,NaN,NaN,NaN
4,Aaron Neville,Everybody Plays The Fool,reminds the listener that everyone falls in lo...,Longing & Heartbreak,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5011,P!nk,Try,about the inevitability of getting hurt when l...,It's Complicated,NaN,It's Complicated,about the inevitability of getting hurt when l...
5012,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything,"about seizing the moment, with a blend of part...",Courtship & Anticipation,NaN,Courtship & Anticipation,"about seizing the moment, with a blend of part..."
5013,Soko,We Might Be Dead By Tomorrow,emphasizing living in the current moment with ...,Longing & Heartbreak,NaN,Longing & Heartbreak,emphasizing living in the current moment with ...
5014,Taylor Swift,Blank Space,satirical view of the media's perception of he...,It's Complicated,NaN,It's Complicated,satirical view of the media's perception of he...


In [199]:
# Ok, similar to before, remove  rows in mjia_update_df (using [performer, song] as the key) that are in love_song_subtypes_df
# then concat love_song_subtypes_df and that result
merged = mjia_updated_df.merge(love_song_subtypes_df, on=['performer', 'song'], how='left', indicator=True)
filtered = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])
filtered
# now rename the justification_x column to justification etc and drop any additional columns
filtered = filtered.rename(columns={'justification_x': 'justification', 'love_song_sub_type_x': 'love_song_sub_type'}).drop(columns=['justification_y', 'love_song_sub_type_y'])
filtered

# now concat the two dataframes
combined = pd.concat([love_song_subtypes_df, filtered], ignore_index=True)
# drop duplicates
combined = combined.drop_duplicates(subset=['performer', 'song'])
combined

# Ok so there was a JSOn "/" issue that created an extra row, but the label is correct so we're good, just drop it
in_both = combined.merge(mjia_updated_df, on=['performer', 'song'], how='left', indicator=True, suffixes=['', '_earlier_mjia'])
in_both = in_both[
    in_both['_merge'] == 'both'
]#.drop(columns=['_merge'])
in_both

final = in_both[['performer', 'song', 'justification', 'love_song_sub_type']]
final


,performer,song,justification,love_song_sub_type
0,10cc,I'm Not In Love,"a repeated denial of being in love, which sugg...",It's Complicated
1,ABBA,Waterloo,"compares being in love with surrender, as if l...",Serenade
2,AWB,Cut The Cake,symbolically about marriage but focuses on the...,NaN
3,Aaliyah,Back & Forth,"a song about having a good time, but not a spe...",NaN
4,Aaron Neville,Everybody Plays The Fool,advising that everyone will be the fool in lov...,Longing & Heartbreak
...,...,...,...,...
5012,P!nk,Try,about the inevitability of getting hurt when l...,It's Complicated
5013,"Pitbull Featuring Ne-Yo, Afrojack & Nayer",Give Me Everything,"about seizing the moment, with a blend of part...",Courtship & Anticipation
5014,Soko,We Might Be Dead By Tomorrow,emphasizing living in the current moment with ...,Longing & Heartbreak
5015,Taylor Swift,Blank Space,satirical view of the media's perception of he...,It's Complicated


In [200]:
# Cleanup the remnant no doubt from my own typo
cleaned = final.copy()
cleaned['love_song_sub_type'] = final.love_song_sub_type.replace({"Courtship & Antisipation": "Courtship & Anticipation"})
cleaned.love_song_sub_type.value_counts()

love_song_sub_type
Serenade                    1032
Longing & Heartbreak         765
It's Complicated             443
Courtship & Anticipation     435
Sexual Conquest              312
Good Riddance                234
Platonic Love                 99
Love Song for the Self        90
Name: count, dtype: int64

In [202]:
assert(prev_df.shape[0] == cleaned.shape[0])
prev_df.love_song_sub_type.value_counts()

love_song_sub_type
Serenade                    1030
Longing & Heartbreak         769
It's Complicated             449
Courtship & Anticipation     430
Sexual Conquest              313
Good Riddance                234
Platonic Love                 99
Love Song for the Self        89
Name: count, dtype: int64

In [203]:
cleaned.to_csv('./data/22.2-OUTPUT-add-mjia-hand-labeling-and-relabel-uncategorized-using-them-as-exaples.csv', index=False)